In [1]:
# A couple of ways to import and slice-dice the fixture list using the premier league API

#imports
import numpy as np
import pandas as pd
import requests
import datetime as dt
import xlwings as xw
from dateutil import tz
import pytz

In [2]:
#pick up the fixture list and the static data (fixtures uses team ids) from the api. Basically lots of JSON files 
url = 'https://fantasy.premierleague.com/api/fixtures/'
r = requests.get(url)
fixtures_json = r.json()

url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
r = requests.get(url)
static_json = r.json()


In [3]:
#from the team static set up a dictionary of team id to team name
teams = static_json["teams"]
team_codes ={team["id"]:team["name"] for team in teams}


In [4]:
#Loop through the fixtures and put in a dataframe. Can't parse the date as it has nulls which causes problems here
fixture_list = [[team_codes[fixture['team_h']],team_codes[fixture['team_a']],fixture['kickoff_time']] for fixture in fixtures_json]


In [5]:
# Attempt 1 - filter the list above by team

#define the team you want
team = "Liverpool"

#tidy up the dataframe and parse the dates (works beter outside the loop)    
dfFixtures = pd.DataFrame(fixture_list,columns=["Home","Away","Date/Time"])
dfFixtures["Date/Time"] = pd.to_datetime(dfFixtures["Date/Time"],format="%Y-%m-%dT%H:%M:%S",utc=True)
#map to local time
dfFixtures["Date/Time"]= dfFixtures["Date/Time"][2:].map(lambda x :x.astimezone(tz.tzlocal()))

teamFixtures = dfFixtures[(dfFixtures["Home"]==team) | (dfFixtures["Away"]==team)]

display(teamFixtures.style.hide_index())

### output to excel. commented out.
#sht = xw.Book().sheets[0]
#sht.range('A1').options(index=False).value = teamFixtures





Home,Away,Date/Time
Norwich,Liverpool,2021-08-14 17:30:00+01:00
Liverpool,Burnley,2021-08-21 12:30:00+01:00
Liverpool,Chelsea,2021-08-28 17:30:00+01:00
Leeds,Liverpool,2021-09-12 16:30:00+01:00
Liverpool,Crystal Palace,2021-09-18 15:00:00+01:00
Brentford,Liverpool,2021-09-25 17:30:00+01:00
Liverpool,Man City,2021-10-03 16:30:00+01:00
Watford,Liverpool,2021-10-16 12:30:00+01:00
Man Utd,Liverpool,2021-10-24 16:30:00+01:00
Liverpool,Brighton,2021-10-30 15:00:00+01:00


In [6]:
# Attempt 2 - create a custom output format and iterate over the fixture list 
#define the team you want

utc = pytz.utc
#first define the function for the custom output text
def format_fixture(x,team):
    home=x[0]
    away=x[1]
    date = utc.localize(dt.datetime.strptime(x[2],"%Y-%m-%dT%H:%M:%SZ")).astimezone(tz.tzlocal())
    if home ==team:
        return ["{}".format(dt.datetime.strftime(date,"%d-%b-%y"))," {}(H) {}".format(away,dt.datetime.strftime(date,"%H:%M"))]
    elif away ==team:
        return ["{}".format(dt.datetime.strftime(date,"%d-%b-%y"))," {}(A) {}".format(home,dt.datetime.strftime(date,"%H:%M"))]

#conditonal formatting function    
def home_bold(val):
    bold = 'bold' if "(H)" in val  else ''
    return 'font-weight:{}'.format(bold)
    
#then put through some pythonic loops to get a dataframe of date and fixture 
def print_fixtures(team):
    output = [format_fixture(x,team) for x in fixture_list[2:] if format_fixture(x,team) != None]

    #print ditching the index and using contitional format
    df = pd.DataFrame(output,columns=["Date","Fixture"])
    return df

In [7]:
sht = xw.Book().sheets[0]
sht.range('d1').options(index=False).value = print_fixtures("Spurs")
sht.range('a1').options(index=False).value = print_fixtures("Liverpool")

In [8]:
print(print_fixtures("Liverpool"))

         Date                   Fixture
0   14-Aug-21          Norwich(A) 17:30
1   21-Aug-21          Burnley(H) 12:30
2   28-Aug-21          Chelsea(H) 17:30
3   12-Sep-21            Leeds(A) 16:30
4   18-Sep-21   Crystal Palace(H) 15:00
5   25-Sep-21        Brentford(A) 17:30
6   03-Oct-21         Man City(H) 16:30
7   16-Oct-21          Watford(A) 12:30
8   24-Oct-21          Man Utd(A) 16:30
9   30-Oct-21         Brighton(H) 15:00
10  06-Nov-21         West Ham(A) 15:00
11  20-Nov-21          Arsenal(H) 15:00
12  27-Nov-21      Southampton(H) 15:00
13  30-Nov-21          Everton(A) 19:45
14  04-Dec-21           Wolves(A) 15:00
15  11-Dec-21      Aston Villa(H) 15:00
16  15-Dec-21        Newcastle(H) 20:00
17  18-Dec-21            Spurs(A) 15:00
18  26-Dec-21            Leeds(H) 15:00
19  28-Dec-21        Leicester(A) 15:00
20  01-Jan-22          Chelsea(A) 15:00
21  15-Jan-22        Brentford(H) 15:00
22  22-Jan-22   Crystal Palace(A) 15:00
23  09-Feb-22        Leicester(H) 20:00


In [9]:
# getting the timezone right
x = dt.datetime.utcnow()
print(x.strftime('%m/%d/%Y %H:%M:%S %Z'))
utc = pytz.utc
x= utc.localize(x)
print(x.strftime('%m/%d/%Y %H:%M:%S %Z'))
x = x.astimezone(tz.tzlocal())
print(x.strftime('%m/%d/%Y %H:%M:%S %Z'))

09/22/2021 19:06:04 
09/22/2021 19:06:04 UTC
09/22/2021 20:06:04 GMT Daylight Time


In [10]:
print(print_fixtures("Spurs"))

         Date                   Fixture
0   15-Aug-21         Man City(H) 16:30
1   22-Aug-21           Wolves(A) 14:00
2   29-Aug-21          Watford(H) 14:00
3   11-Sep-21   Crystal Palace(A) 12:30
4   19-Sep-21          Chelsea(H) 16:30
5   26-Sep-21          Arsenal(A) 16:30
6   03-Oct-21      Aston Villa(H) 14:00
7   17-Oct-21        Newcastle(A) 16:30
8   24-Oct-21         West Ham(A) 14:00
9   30-Oct-21          Man Utd(H) 17:30
10  06-Nov-21          Everton(A) 15:00
11  20-Nov-21            Leeds(H) 15:00
12  27-Nov-21          Burnley(A) 15:00
13  01-Dec-21        Brentford(H) 19:45
14  04-Dec-21          Norwich(H) 15:00
15  11-Dec-21         Brighton(A) 15:00
16  14-Dec-21        Leicester(A) 19:45
17  18-Dec-21        Liverpool(H) 15:00
18  26-Dec-21   Crystal Palace(H) 15:00
19  28-Dec-21      Southampton(A) 15:00
20  01-Jan-22          Watford(A) 15:00
21  15-Jan-22          Arsenal(H) 15:00
22  22-Jan-22          Chelsea(A) 15:00
23  09-Feb-22      Southampton(H) 19:45
